In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 13)


In [4]:
genes_to_look = ['MGAT3','ID1']

In [5]:
bed_files = []
signal_files = []

In [8]:
serv = '/mnt/jabba'

out_dir = "/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR"
out_dir_save = os.path.join(out_dir,"Results/Figures/Figure1")
if not os.path.exists(out_dir_save):
    os.mkdir(out_dir_save)

In [7]:
tss = pd.read_csv("Results/output/TSS1.bed",sep="\t",header=None)
tss.rename({0:"Chr",1:"Start",2:"End",3:"ID",4:"Stat",5:"Strand"},axis=1,inplace=True)
tss.head()

,Chr,Start,End,ID,Stat,Strand
0,NW_020822366.1,724533,724683,p1@Bmp4_XM_027386528.1,4.074290,-
1,NW_020822366.1,1085431,1085581,p1@Cdkn3_XM_027386531.1,1.212188,+
2,NW_020822366.1,1085743,1085745,p0@Cdkn3_XM_027386533.1,0.000000,+
3,NW_020822366.1,1113842,1113992,p2@Cnih1_XM_027386534.1,1.049218,-
4,NW_020822366.1,1114280,1114430,p1@Cnih1_XM_027386534.1,1.527630,-


In [9]:
vary_width = [0,100,1000,10000]

## For Publication:
-ATAC
-GROCap
-GRO
-START
-STARTinput
-TSS
-RefSeq
-dTSS

In [20]:
curr_folder = '/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/'
igv_script = """new
genome {serv}{ref_fa}
load {serv}{annotation}

load {serv}{out_dir}/Results/output/TSS1.bed
load {serv}{out_dir}/Results/merged/tags_TSS_merged.bedgraph.gz

# CHO
load {serv}{data_folder}/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig
load {serv}{data_folder}/CHO/GROCap/bedgraph_files/trim_CHO_GRO1.bedgraph.gz 
load {serv}{data_folder}/CHO/GROCap/bedgraph_files/trim_CHO_GROCap1.bedgraph.gz 

load {serv}{data_folder}/CHO/START/bedgraph_files/trim_CHO_START1.bedgraph.gz 
load {serv}{data_folder}/CHO/START/bedgraph_files/trim_CHO_STARTinput1.bedgraph.gz 

# Brain
load {serv}{data_folder}/Brain/ATAC/out/signal/macs2/rep1/Brain_ATAC1.trim.nodup.tn5.pf.pval.signal.bigwig
load {serv}{data_folder}/Brain/GRO/bedgraph_files/trim_Brain_GROCap1.bedgraph.gz 
load {serv}{data_folder}/Brain/GRO/bedgraph_files/trim_Brain_GRO1.bedgraph.gz 

#Liver 
load {serv}{data_folder}/Liver/GRO/bedgraph_files/trim_Liver_GROCap1.bedgraph.gz
load {serv}{data_folder}/Liver/ATAC/out/signal/macs2/rep1/Liver_ATAC1.trim.nodup.tn5.pf.pval.signal.bigwig

snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1""".format(
    serv=serv,
    data_folder=data_folder,
    out_dir=out_dir,
    ref_fa=ref_fa,
    annotation=annotation)

snaps = "snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1".format(
    serv=serv, out_dir=out_dir)

for ind, val in tss[tss["ID"].str.upper().str.contains("MGAT3")].iterrows():
    for w in vary_width:
        snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}_{w}\n'.format(
            chrom=tss.loc[ind, "Chr"],
            start=tss.loc[ind, "Start"] - w,
            end=tss.loc[ind, "End"] + w,
            name=tss.loc[ind, "ID"],
            w=w)

# for i in glob.glob("f04_peaks_bed/*bed"):
#     igv_script = igv_script +  "\nload " + serv + os.path.join(curr_folder, i)
if not os.path.exists(os.path.join(out_dir_save,"bed_files/")):
    os.mkdir(os.path.join(out_dir_save,"bed_files/"))
    
for t in tissues:
    for s in ["GROCap", "START"]:
        for i in glob.glob(
                os.path.join(data_folder, t, s, "bed_files/", "*merge*bed")):
            name = os.path.join(out_dir_save,"bed_files/" + t + "_" + s + ".bed")
            if not os.path.exists(name):
                cmd = "cp {i} {name}".format(i=i,name=name)
                !{cmd}
            #igv_script = os.path.join(s)
            igv_script = igv_script + "\nload " + serv + name

with open('Results/Figures/Figure1/igv_batch_TSS_Pub_LOAD.bat', 'w') as f:
    f.write(igv_script)

batch = igv_script + "\n" + snaps

with open('Results/Figures/Figure1/igv_batch_TSS_MGAT3.bat', 'w') as f:
    f.write(batch)

with open('Results/Figures/Figure1/igv_batch_TSS_MGAT3_SNAP.bat', 'w') as f:
    f.write(snaps)

batch

'new\ngenome /mnt/jabba/data/isshamie/genome/ncbi_anno_103/GCF_003668045.1_CriGri-PICR_genomic.fna\nload /mnt/jabba/data/isshamie/genome/ncbi_anno_103/alt_CriGri-PICR_top_level.gff3\n\nload /mnt/jabba/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/Analysis_NCBI_PICR/Results/merged/tags_TSS_merged.bedgraph.gz\n\n# CHO\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/GROCap/bedgraph_files/trim_CHO_GRO1.bedgraph.gz \nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/GROCap/bedgraph_files/trim_CHO_GROCap1.bedgraph.gz \n\nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/START/bedgraph_files/trim_CHO_START1.bedgraph.gz \nload /mnt/jabba/data/isshamie/TSS/NCBI_PICR_103_Processed//CHO/START/bedgraph_files/trim_CHO_STARTi

In [17]:
tss[tss["ID"].str.upper().str.contains("MGAT5")]

,Chr,Start,End,ID,Stat,Strand
31722,NW_020822567.1,39812140,39812290,p2@Mgat5_XM_027417769.1,0.453318,-
31723,NW_020822567.1,39813270,39813420,p1@Mgat5_XM_027417769.1,1.723456,-
39455,NW_020822611.1,4879714,4879864,p1@Mgat5b_XM_027425561.1,0.678518,-


In [18]:
tss[tss["ID"].str.upper().str.contains("MGAT3")]

,Chr,Start,End,ID,Stat,Strand
18038,NW_020822470.1,7502049,7502199,p1@Mgat3_NM_001244074.1,0.394452,+


----------

----------

### Random gene TSS regions

In [17]:
snaps = 'snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1\n'.format(serv=serv)
for i in range(len(tss)):
    if i % 2500 == 0:
        for w in vary_width:
            snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}_{w}\n'.format(
                chrom=tss.loc[i, "Chr"],
                start=tss.loc[i, "Start"] - w,
                end=tss.loc[i, "End"] + w,
                name=tss.loc[i, "ID"],
                w=w)
batch = igv_script + "\n" + snaps
with open('Results/Figures/Figure1/igv_batch_TSS.bat', 'w') as f:
    f.write(batch)

with open('Results/Figures/Figure1/igv_batch_TSS_SNAP.bat', 'w') as f:
    f.write(snaps)
    
batch



'new\ngenome /mnt/jabba/data/isshamie/genome/picr_final/picr.fa\nload /mnt/jabba/data/isshamie/genome/picr_final/final.gff3\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_TSS_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_input_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/1015_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\n\nsnapshotDirectory /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/igv_snapshots\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/FemaleReproductive_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Spleen_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Kidney_GROCap.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Resul

### Second: Load every single peak file and the final output and see the shift
Have a CHO ATAC and GRO as well

In [14]:
curr_folder = '{out_dir}'
igv_script = """new
genome {serv}{ref_fa}
load {serv}{annotation}
load {serv}{out_dir}/Results/output/TSS1.bed
load {serv}{out_dir}/Results/merged/tags_TSS_merged.bedgraph.gz
load {serv}{out_dir}/Results/merged/tags_input_merged.bedgraph.gz
load {serv}{data_folder}/CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig

snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1""".format(serv=serv,data_folder=data_folder)

for i in glob.glob("f04_peaks_bed/*bed"):
    igv_script = igv_script +  "\nload " + serv + os.path.join(curr_folder, i)
    
igv_script

'new\ngenome /mnt/jabba/data/isshamie/genome/picr_final/picr.fa\nload /mnt/jabba/data/isshamie/genome/picr_final/final.gff3\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/output/TSS1.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_TSS_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/merged/tags_input_merged.bedgraph.gz\nload /mnt/jabba/data/isshamie/TSS/1015_Processed//CHO/ATAC/out/signal/macs2/pooled_rep/CHO_ATAC1.trim.nodup.tn5_CHO_ATAC3.trim.nodup.tn5.pf.pval.signal.bigwig\n\nsnapshotDirectory /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/Results/igv_snapshots\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/FemaleReproductive_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Spleen_START.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Results/f04_peaks_bed/Kidney_GROCap.bed\nload /mnt/jabba/data/isshamie/TSS/Analysis/10_15_Resul

### MGAT regions

In [16]:
snaps = 'snapshotDirectory {serv}{out_dir}/Results/Figures/Figure1\n'.format(serv=serv)


for ind,val in tss[tss["ID"].str.contains("MGAT")].iterrows():
    
    for w in vary_width:
        snaps = snaps + 'goto {chrom}:{start}-{end}\nsort\ncollapse\nsnapshot {name}_{w}\n'.format(
            chrom=tss.loc[ind, "Chr"],
            start=tss.loc[ind, "Start"] - w,
            end=tss.loc[ind, "End"] + w,
            name=tss.loc[ind, "ID"],
            w=w)

batch = igv_script + "\n" + snaps
with open('Results/Figures/Figure1/igv_batch_TSS_MGAT.bat', 'w') as f:
    f.write(batch)
batch


with open('Results/Figures/Figure1/igv_batch_TSS_MGAT_SNAP.bat', 'w') as f:
    f.write(snaps)